In [1]:
# Load in packages for pandas, astropy, etc. 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.table import Table
from astropy.io import ascii
from astropy.table import Column, MaskedColumn
from astropy.io.ascii import masked
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.cosmology import LambdaCDM 
from astroquery.simbad import Simbad
from astroquery.sdss import SDSS
from astropy.coordinates import match_coordinates_sky
import os 
import json 

cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7) #Creating our choice of cosmology here...

pd.set_option('display.max_columns', 300) # Setting max number of rows per df to be the size of the df
pd.set_option('display.max_rows', None)




In [2]:
# here we are working in a new notebook to make things a bit cleaner and easier

# we'll be working on formatting the analysis and confirmation methods columns in this notebook, reading in the \
# formatted version from the formatting_DR0p5 notebook.

# as of Feb 6/7, I am now keeping track of the various changes I make to the confirmation and analysis methods \
# columns....

# just added optical imaging and infrared iamging to sekiguchi selection method

# just added optical fiber spectroscopy and optical spectroscopy to the selection method in the double-peaked \
# emission line notebooks. 
# Binary broad line and offset line notebook looks good. 
# We need to check the binary notebooks (and other notebooks) and make sure we have all of these noted \
# appropriately!

# fixed the missing "2016" in the offset AGN table for the Muller-Sanchez objects

# added 'optical imaging' to teh selection methods for the binary periodicity tables of Graham and Charisi

# added 'radio imaging' in the x-shaped radio source notebook for the selection method. Also ensured systype \
# was always flagged as binary agn and recoil

# added optical imaging and IFU optical imaging to Fu+2018, optical imaging to Fu+2015 select method, and \
# and radio double sources to Fu+2015 confirm method

# took optical photometry out of hennawi2010 select method 
# removed optical photometry from myers2007 method

# added optical spectroscopy to confirm methods for findlay 2018 along with optical imaging and id-ir imaging to \
# the selection methods 

# just added mid-IR imaging to the rusu+2019 objects

# added optical slit spectroscopy to lemon+2018 targets

# added mid-IR imaging to agnello list and anguita list

# in the individual targets csv, I've now gone through and added MIR-Imaging to instances where we had Mid-IR \
# Colors ( or Infrared Imaging wherever else applicable). I also went through and tried to add in Optical \
# Spectroscopy wherever we had optical spectroscopic techniques but did not yet include the tag for optical \
# spectroscopy. I think I've gotten everything, but we should probably add some code to include it if there \
# are rows where we still have that problem.

# 'Photometry' was removed from the individual target table (only 4 instances)
# Adjusted some other stuff in the individual target table, including adding optical imaging to objects that  \
# I KNOW needed that as a selection technique, added X-ray imaging and spectroscopy along with near-IR spec \
# for the mid-IR selected targets from pfeifle and satyapal, and adjusted mid-IR colors to INfrared lumunosity for \
# the LIRG selected objects

#************
# we'll need to manually add X-ray imaging and X-ray spectroscopy to the vignali 2018 target analysis methods


# Update: 19 June 2024
# Updated selection methods for Lena+2014 targets (removed near-IR imaging for most targets since optical was\
# more important)

# Just noticed that the selection method listed for Barrows+2016 is not correct. Correcting now. Selection \
# method now covers optical spec/fiber spec/spec emission line ratios/optical imaging

# Added optical imaging to selection method for Kim+2017 (the offset recoil AGN candidate)

# Added optical imaging and opticacl positional offset for menezes+2014 and 2016

# Added Radio double sources for selection of CSO 0402



In [3]:
# reading in the formatted version DR0p8
MAC = pd.read_csv('MAC_DR0p8_formatted.csv', sep=',')


In [4]:
# TO DO:

# in 3C 321, we need to add Spectropolarimetry to analysis methods. we removed it from the confirmation methods

# in SDSS J233646.2-010732.6,SDSS J2336-0107A, we need to add Centroid Shift Analysis to analysis methods. \
# removed it from confirmation methods

# we'll need to manually add X-ray imaging and X-ray spectroscopy to the vignali 2018 target analysis methods




In [5]:
#MAC

# Here we are formatting the selection method column.
# Note, this was originally in Dr0p5 formatting notebook, but has been added here to better organize the notebooks and streamline things

In [6]:
# this was moved from DR0p5 formatting notebook to here on 4 April 2024
# Here we're going to work on reformatting some of the phrases used for the selection techniques


types = MAC['Selection Method'].dropna().str.split(' / ')

# Step 2 and 3: Remove duplicates, alphabetize, and replace 'binary quasar' with 'dual AGN' in any context for each cell
def process_cell(cell):
    cell = [x.replace('Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Optical Optical Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Fiber Optical Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Slit Optical Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Long-Slit Optical Spectroscopy', 'Optical Long-Slit Spectroscopy') for x in cell]
    cell = [x.replace('IFU Optical Spectroscopy', 'Optical IFU Spectroscopy') for x in cell]
    cell = [x.replace('Double Radio Sources', 'Radio Double Sources') for x in cell]
    cell = [x.replace('IFU Optical Imaging', 'Optical IFU Imaging') for x in cell]
    cell = [x.replace('X-Shaped Radio Source', 'Radio X-Shaped Source') for x in cell]
    cell = [x.replace('LOS Radial Velocity Shifts', 'Optical LOS Radial Velocity Shifts') for x in cell]
    cell = [x.replace('Optical Optical LOS Radial Velocity Shifts', 'Optical LOS Radial Velocity Shifts') for x in cell]
    cell = [x.replace('Velocity Offset Broad Optical Spectroscopic Emission Lines', 'Optical Velocity Offset Broad Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Double-Peaked Optical Spectroscopic Emission Lines', 'Optical Double-Peaked Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Double-Peaked Broad Optical Spectroscopic Emission Lines', 'Optical Double-Peaked Broad Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Velocity Offset Narrow Optical Spectroscopic Emission Lines', 'Optical Velocity Offset Narrow Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Double-Peaked Narrow UV Spectroscopic Emission Lines', 'UV Double-Peaked Narrow Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Varstrometry', 'Optical Varstrometry') for x in cell]
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

# Step 4: Join the contents back into a single string
MAC['Processed Selection Method'] = processed_types.apply(' / '.join)




In [7]:
## this was moved from the dr0p5 formatting notebook to here on 4 April 2024
# commenting this out because this code is used again below after some corrections are made...
#import re
#
#def parse_selection_method(s, wavebands, compound_wavebands):
#    # Split the string by slashes
#    methods = s.split(' / ')
#
#    result_dict = {}
#    for method in methods:
#        # Explicitly handle 'Optical-Mid-IR Colors'
#        if 'Optical-Mid-IR Colors' in method:
#            for wb in ['Optical', 'Mid-IR']:
#                if wb in result_dict:
#                    result_dict[wb].append('Optical-Mid-IR Colors')
#                else:
#                    result_dict[wb] = ['Optical-Mid-IR Colors']
#        else:
#            # First check for compound wavebands
#            compound_processed = False
#            for compound in compound_wavebands:
#                if compound in method:
#                    term = method.replace(compound, '').strip()
#                    for wb in compound.split('-'):
#                        if wb in result_dict:
#                            result_dict[wb].append(compound + ' ' + term)
#                        else:
#                            result_dict[wb] = [compound + ' ' + term]
#                    compound_processed = True
#                    break
#            
#            if not compound_processed:
#                # Process standard wavebands
#                for waveband in wavebands:
#                    if waveband in method:
#                        term_start_idx = method.index(waveband) + len(waveband)
#                        term = method[term_start_idx:].strip()
#
#                        if waveband in result_dict:
#                            result_dict[waveband].append(term)
#                        else:
#                            result_dict[waveband] = [term]
#
#    return result_dict
#
## List of known wavebands
#known_wavebands = ['Optical', 'Mid-IR', 'Radio', 'Hard X-ray', 'X-ray', 'UV', 'Infrared', 'Submillimeter',\
#                   'Near-IR','Gamma-Ray']
#
## List of compound wavebands
#compound_wavebands = ['Radio-Optical', 'Optical-Mid-IR', 'Radio-Infrared']
#
## Apply the function to each row
#MAC['Parsed Selection Method'] = MAC['Processed Selection Method'].apply(lambda x: parse_selection_method(x, known_wavebands, compound_wavebands))



In [8]:
# here we need to go back and start correcting selection methods that have missed things

# here we're going to correct all of the periodicity and quasi-periodicity listings...
tech = ['Gamma-Ray Periodicity','Gamma-Ray Quasi-Periodicity','Optical Periodicity','Optical Quasi-Periodicity',\
       'X-ray Periodicity','X-ray Quasi-Periodicity','Near-IR Periodicity','Near-IR Quasi-Periodicity',\
       'Hard X-ray Periodicity','Hard X-ray Quasi-Periodicity','Radio Periodicity','Radio Quasi-Periodicity']

techadd = ['Gamma-Ray Imaging','Gamma-Ray Imaging','Optical Imaging','Optical Imaging','X-ray Imaging',\
           'X-ray Imaging','Near-IR Imaging','Near-IR Imaging','Hard X-ray Imaging','Hard X-ray Imaging',\
           'Radio Imaging','Radio Imaging']
count = 0
for i, j in zip(tech,techadd):
    for index, row in MAC.iterrows():
        if (str(i) in row['Processed Selection Method']) and (str(j) not in row['Processed Selection Method']):
            count += 1
            MAC.at[index, 'Processed Selection Method'] += ' / ' + str(j)

print(str(count)+" instances corrected!")

# now focusing on various color selection techniques and radio imaging issues
tech = ['Mid-IR Colors','Radio Jet Precession','Radio Double Jet','Radio Double Sources',\
       'Optical Varstrometry','Near-IR Positional Offset','Optical Astrometry','X-ray Positional Offset',\
       'Near-IR Slit Spectroscopy','Optical Positional Offset','Optical IFU Imaging']

techadd = ['Mid-IR Imaging','Radio Imaging','Radio Imaging','Radio Imaging','Optical Imaging',\
           'Near-IR Imaging','Optical Imaging','X-ray Imaging','Near-IR Spectroscopy','Optical Imaging',\
           'Optical Imaging']

count = 0
for i, j in zip(tech,techadd):
    for index, row in MAC.iterrows():
        if (str(i) in row['Processed Selection Method']) and (str(j) not in row['Processed Selection Method']):
            count += 1
            MAC.at[index, 'Processed Selection Method'] += ' / ' + str(j)

# we're good on optical and near-IR colors/imaging. It seems I corrected all of that
print(str(count)+" instances corrected!")

# and here we're going to fix any remaining issues with spectroscopy
tech = ['Optical Fiber Spectroscopy','Optical Long-Slit Spectroscopy','Optical LOS Radial Velocity Shifts',\
        'Optical Slit Spectroscopy','Velocity Offset Emission Lines',\
        'Optical Velocity Offset Narrow Spectroscopic Emission Lines','Optical Slitless Spectroscopy',\
        'Optical Velocity Offset Broad Spectroscopic Emission Lines','Optical Spectroscopic Emission Line Ratios',\
        'Optical IFU Spectroscopy','Optical Double-Peaked Spectroscopic Emission Lines',\
        'Optical Double-Peaked Broad Spectroscopic Emission Lines']

count = 0
for i in tech:
    for index, row in MAC.iterrows():
        if (str(i) in row['Processed Selection Method']) and ('Optical Spectroscopy' not in row['Processed Selection Method']):
            count += 1
            MAC.at[index, 'Processed Selection Method'] += ' / Optical Spectroscopy'

print(str(count)+" instances of optical spectroscopy fixed!")

        

62 instances corrected!
30 instances corrected!
1 instances of optical spectroscopy fixed!


In [9]:
count=0
for index, row in MAC.iterrows():
    if ('Optical IFU Imaging' in row['Processed Selection Method']) and ('Optical Imaging' not in row['Processed Selection Method']):
        count += 1
        #print(row)
        
print(count)

0


In [10]:
methods = MAC['Processed Selection Method'].dropna().str.split(' / ')

# Flatten the lists and find unique labels
unique_methods = set(method for sublist in methods for method in sublist)

print("Unique selection methods:", unique_methods)
print(len(unique_methods))



Unique selection methods: {'Optical Imaging', 'Optical Periodicity', 'Optical Colors', 'Optical IFU Imaging', 'Near-IR Positional Offset', 'Near-IR Spectroscopy', 'Optical Varstrometry', 'Infrared Luminosity', 'Near-IR Periodicity', 'Optical Velocity Offset Narrow Spectroscopic Emission Lines', 'Optical Spectroscopy', 'X-ray Positional Offset', 'Optical-Mid-IR Colors', 'Optical Slit Spectroscopy', 'Radio Periodicity', 'Near-IR Slit Spectroscopy', 'Radio-Infrared Colors', 'UV Double-Peaked Narrow Spectroscopic Emission Lines', 'Optical Double-Peaked Spectroscopic Emission Lines', 'Radio Double Jet', 'Optical Long-Slit Spectroscopy', 'Optical Astrometry', 'Optical IFU Spectroscopy', 'Gamma-Ray Quasi-Periodicity', 'Optical LOS Radial Velocity Shifts', 'Radio X-Shaped Source', 'Optical Quasi-Periodicity', 'Radio-Optical Positional Offsets', 'Near-IR Colors', 'Optical Fiber Spectroscopy', 'X-ray Imaging', 'Gamma-Ray Periodicity', 'X-ray Periodicity', 'Optical Slitless Spectroscopy', 'Radio 

In [11]:
import re

def parse_selection_method(s, wavebands, compound_wavebands):
    # Split the string by slashes
    methods = s.split(' / ')

    result_dict = {}
    for method in methods:
        # Explicitly handle 'Optical-Mid-IR Colors'
        if 'Optical-Mid-IR Colors' in method:
            for wb in ['Optical', 'Mid-IR']:
                if wb in result_dict:
                    result_dict[wb].append('Optical-Mid-IR Colors')
                else:
                    result_dict[wb] = ['Optical-Mid-IR Colors']
        else:
            # First check for compound wavebands
            compound_processed = False
            for compound in compound_wavebands:
                if compound in method:
                    term = method.replace(compound, '').strip()
                    for wb in compound.split('-'):
                        if wb in result_dict:
                            result_dict[wb].append(compound + ' ' + term)
                        else:
                            result_dict[wb] = [compound + ' ' + term]
                    compound_processed = True
                    break
            
            if not compound_processed:
                # Process standard wavebands
                for waveband in wavebands:
                    if waveband in method:
                        term_start_idx = method.index(waveband) + len(waveband)
                        term = method[term_start_idx:].strip()

                        if waveband in result_dict:
                            result_dict[waveband].append(term)
                        else:
                            result_dict[waveband] = [term]

    return result_dict

# List of known wavebands
known_wavebands = ['Optical', 'Mid-IR', 'Radio', 'Hard X-ray', 'X-ray', 'UV', 'Infrared', 'Submillimeter',\
                   'Near-IR','Gamma-Ray']

# List of compound wavebands
compound_wavebands = ['Radio-Optical', 'Optical-Mid-IR', 'Radio-Infrared']

# Apply the function to each row
MAC['Parsed Selection Method'] = MAC['Processed Selection Method'].apply(lambda x: parse_selection_method(x, known_wavebands, compound_wavebands))



# Now we're going to work on correcting/formatting the analysis and confirmation methods, and generate the parsed, nested lists of dictionaries for each

In [12]:
# first, we're going to generate the analysis column and populate it, and then we'll make some corrections \
# to both the analysis column and confirmation column

# NOTE: THIS CONTENT WAS ORIGINALLY IN THE FORMATTING DR0P5 NOTEBOOK BUT HAS BEEN ADDED HERE TO STREAMLINE AND \
# ORGANIZE THINGS
xliu = pd.read_csv('xliu2011_formated_catalog_forrflags.csv', sep=',')
xliu_agns = xliu[((xliu['FAGN1']==0) | (xliu['FAGN1']==3) | (xliu['FAGN1']==4) | (xliu['FAGN1']==5)) & \
                 ((xliu['FAGN2']==0) | (xliu['FAGN2']==3) | (xliu['FAGN2']==4) | (xliu['FAGN2']==5)) & \
                 (xliu['Sep']>10)]

#len(xliu_agns)
xliu_agns_1list = xliu_agns['Name1'].to_list()
xliu_agns_2list = xliu_agns['Name2'].to_list()


MAC['Analysis Method'] = MAC['Selection Method']

#xliu_agns_1list = xliu_agns['Name1'].to_list()
#xliu_agns_2list = xliu_agns['Name2'].to_list()

for index, row in MAC.iterrows():
    if row['Name1'] in xliu_agns_1list:
        #print('True')
        MAC.at[index, 'Confirmation Method'] = MAC.at[index, 'Selection Method']
        MAC.at[index, 'Primary System Type'] = 'Dual AGN'
        
for index, row in MAC.iterrows():
    if (row['Confidence Flag'] == 1) and ((row['Confirmation Method']!='-99') or (row['Confirmation Method']!=-99)):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])
    elif (row['Confidence Flag'] == 0.5) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'
    elif (row['Confidence Flag'] == 0.0) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'    
    elif (row['Confidence Flag'] == -0.5) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'   
    elif (row['Confidence Flag'] == -1) and (row['Confirmation Method']!='-99'):
        MAC.at[index, 'Analysis Method'] += ' / ' + str(MAC.at[index, 'Confirmation Method'])   
        MAC.at[index, 'Confirmation Method'] = '-99'   

# manually adjusting the confirmation strats on a few targets
papers = ['Inada+2010 ; Gattano+2014','Inada+2012 ; Eftekharzadeh+2017','Inada+2012','Dutta+2018 ; Keel+2019']
for i in papers:
    for index, row in MAC.iterrows():
        if (row['Paper(s)']==str(i)) and (row['Confidence Flag'] == 1) and ((row['Confirmation Method']=='-99') or row['Confirmation Method']==-99):
            MAC.at[index, 'Confirmation Method'] = str(MAC.at[index, 'Selection Method'])


# more manual adjustments to confirmation strategies:
objs = ['J100602.14+071131.0','J102325.57+324348.4','J115822.58+323102.2','J162345.20+080851.1','IC 5338']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == 1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Radio Imaging'

# and down below here we'll need to make 'Dual AGN Candidate' --> 'Dual AGN' for confidence flags of 1




In [13]:
# making manual adjustments to confirmation strategies for objects that we've flagged as -1:
objs = ['J101847.57+294114.1','J105553.64+152027.5','J111729.22+614015.2','J134640.79+522836.5',\
        '013412.78-010729.6','222051.44+005815.0','Mrk 78','J112939.77+605742.5']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == -1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Radio Imaging'

objs = ['SDSS J0914+0853']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == -1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'X-ray Imaging / X-ray Spectroscopy'

objs = ['SDSS J0818+0601A']
for i in objs:
    for index, row in MAC.iterrows():
        if (row['Name1']==str(i)) and (row['Confidence Flag'] == 1) and (row['Confirmation Method']=='-99'):
            MAC.at[index, 'Confirmation Method'] = 'Optical Imaging / Optical Spectroscopy'



# Now working specifically on analysis methods here

In [14]:
# now we'll work to format the analysis methodology cell

types = MAC['Analysis Method'].dropna().str.split(' / ')

# Step 2 and 3: Remove duplicates, alphabetize, and replace 'binary quasar' with 'dual AGN' in any context for each cell
def process_cell(cell):
    cell = [x.replace('Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Optical Optical Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Double-Peaked Optical Spectroscopic Emission Lines', 'Optical Double-Peaked Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Velocity Offset Narrow Optical Spectroscopic Emission Lines', 'Optical Velocity Offset Narrow Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('NIR Imaging', 'Near-IR Imaging') for x in cell]
    cell = [x.replace('Slit Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Optical Optical Slit Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Slit Optical Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Optical Long-Optical Slit Spectroscopy', 'Optical Long-Slit Spectroscopy') for x in cell] 
    cell = [x.replace('Long-Slit Optical Spectroscopy', 'Optical Long-Slit Spectroscopy') for x in cell]
    cell = [x.replace('IFU Optical Spectroscopy', 'Optical IFU Spectroscopy') for x in cell]
    cell = [x.replace('Optical Spectroscopy Emission Line Ratios', 'Optical Spectroscopic Emission Line Ratios') for x in cell]
    cell = [x.replace('Fiber Optical Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('X-Shaped Radio Source', 'Radio X-Shaped Source') for x in cell]
    cell = [x.replace('Varstrometry', 'Optical Varstrometry') for x in cell]
    cell = [x.replace('Astrometry', 'Optical Astrometry') for x in cell]
    cell = [x.replace('Optical Optical Astrometry', 'Optical Astrometry') for x in cell]
    cell = [x.replace('Double-Peaked Broad Optical Spectroscopic Emission Lines', 'Optical Double-Peaked Broad Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Velocity Offset Broad Optical Spectroscopic Emission Lines', 'Optical Velocity Offset Broad Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('Double-Peaked Narrow UV Spectroscopic Emission Lines', 'UV Double-Peaked Narrow Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('LOS Radial Velocity Shifts', 'Optical LOS Radial Velocity Shifts') for x in cell]
    cell = [x.replace('Optical Optical LOS Radial Velocity Shifts', 'Optical LOS Radial Velocity Shifts') for x in cell]
    cell = [x.replace('  ', ' ') for x in cell]
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

MAC['Processed Analysis Method'] = processed_types.apply(' / '.join)


In [15]:
#
#for i in unique_methods:
#    num = 0
#    for index,row in MAC.iterrows():
#        if i in row['Processed Analysis Method']:
#            num += 1
#        else:
#            num += 0
#    print("Method "+str(i)+" identified "+str(num)+" times!")
#
#


In [16]:
# displaying the unique analysis methods in a list here
methods = MAC['Processed Analysis Method'].dropna().str.split(' / ')

# Flatten the lists and find unique labels
unique_methods = set(method for sublist in methods for method in sublist)

print("Unique Analysis methods:", unique_methods)
print(len(unique_methods))

Unique Analysis methods: {'Optical Imaging', ' Optical Slit Spectroscopy', '-99', 'Near-IR Periodicity', 'Near-UV Spectroscopy', 'X-ray Positional Offset', 'Optical Slit Spectroscopy', 'Millimeter Quasi-Periodicity', 'Hard X-ray Spectroscopy', 'Radio X-Shaped Source', 'Optical Slitless Spectroscopy', 'UV Spectroscopy', 'IFU Optical Imaging', 'Optical Positional Offset', 'Hard X-ray BAT Selection', 'Optical Velocity Offset Broad Spectroscopic Emission Lines', 'Optical IFU Imaging', 'Radio Periodicity', 'Optical Double-Peaked Spectroscopic Emission Lines', 'Optical Astrometry', 'Optical imaging', 'Optical IFU Spectroscopy', 'Gamma-Ray Quasi-Periodicity', 'nan', 'Radio-Optical Positional Offsets', 'Radio Double Sources', 'Velocity Offset Emission Lines', 'Submillimeter Imaging', 'Near-IR IFU Imaging', 'Radio Jet Precession', 'Mid-IR Imaging', 'Optical Spectroscopic Emission Line Ratios', 'Double Radio Sources', 'Optical Periodicity', 'Optical Colors', 'Near-IR Spectroscopy', 'Optical Vars

In [17]:
# displaying here the number of instances for each analysis method
for i in unique_methods:
    num = 0
    for index,row in MAC.iterrows():
        if i in row['Processed Analysis Method']:
            num += 1
        else:
            num += 0
    print("Method "+str(i)+" identified "+str(num)+" times!")




Method Optical Imaging identified 931 times!
Method  Optical Slit Spectroscopy identified 86 times!
Method -99 identified 33 times!
Method Near-IR Periodicity identified 1 times!
Method Near-UV Spectroscopy identified 1 times!
Method X-ray Positional Offset identified 3 times!
Method Optical Slit Spectroscopy identified 131 times!
Method Millimeter Quasi-Periodicity identified 1 times!
Method Hard X-ray Spectroscopy identified 1 times!
Method Radio X-Shaped Source identified 526 times!
Method Optical Slitless Spectroscopy identified 2 times!
Method UV Spectroscopy identified 7 times!
Method IFU Optical Imaging identified 16 times!
Method Optical Positional Offset identified 61 times!
Method Hard X-ray BAT Selection identified 12 times!
Method Optical Velocity Offset Broad Spectroscopic Emission Lines identified 572 times!
Method Optical IFU Imaging identified 8 times!
Method Radio Periodicity identified 10 times!
Method Optical Double-Peaked Spectroscopic Emission Lines identified 1824

# Now working on formatting confirmation methods

In [18]:
# and here we're working on confirmation methods....
types = MAC['Confirmation Method'].str.split(' / ')

# Step 2 and 3: Remove duplicates, alphabetize, and replace 'binary quasar' with 'dual AGN' in any context for each cell
def process_cell(cell):
    cell = [x.replace('Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Optical Optical Fiber Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Fiber Optical Spectroscopy', 'Optical Fiber Spectroscopy') for x in cell]
    cell = [x.replace('Long-Slit Optical Spectroscopy', 'Optical Long-Slit Spectroscopy') for x in cell]
    cell = [x.replace('Slit Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Optical Optical Slit Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Slit Optical Spectroscopy', 'Optical Slit Spectroscopy') for x in cell]
    cell = [x.replace('Optical Long-Optical Slit Spectroscopy', 'Optical Long-Slit Spectroscopy') for x in cell] 
    cell = [x.replace('IFU Optical Spectroscopy', 'Optical IFU Spectroscopy') for x in cell]
    cell = [x.replace('Double Radio Sources', 'Radio Double Sources') for x in cell]
    cell = [x.replace('IFU Optical Imaging', 'Optical IFU Imaging') for x in cell]
    cell = [x.replace('NIR Imaging', 'Near-IR Imaging') for x in cell]
    cell = [x.replace('Optical Optical LOS Radial Velocity Shifts', 'Optical LOS Radial Velocity Shifts') for x in cell]
    cell = [x.replace('X-Shaped Radio Source', 'Radio X-Shaped Source') for x in cell]
    cell = [x.replace('Double-Peaked Optical Spectroscopic Emission Lines', 'Optical Double-Peaked Spectroscopic Emission Lines') for x in cell]
    cell = [x.replace('  ', ' ') for x in cell]
    # Remove duplicates using set and then convert back to list
    unique_labels = list(set(cell))
    # Alphabetize the contents
    unique_labels.sort()
    return unique_labels

processed_types = types.apply(process_cell)

MAC['Processed Confirmation Method'] = processed_types.apply(' / '.join)
# Display the updated DataFrame
#print(MAC[['Processed Selection Method']])

In [19]:
# displaying now the unique confirmation methods in a list
methods = MAC['Processed Confirmation Method'].dropna().str.split(' / ')

# Flatten the lists and find unique labels
unique_methods = set(method for sublist in methods for method in sublist)

print("Unique Confirmation methods:", unique_methods)
print(len(unique_methods))

Unique Confirmation methods: {'Optical Imaging', 'Optical Colors', 'Optical IFU Imaging', 'Near-IR Spectroscopy', ' Optical Slit Spectroscopy', '-99', 'Optical Spectroscopy', 'Near-UV Spectroscopy', 'Optical Slit Spectroscopy', 'Optical Double-Peaked Spectroscopic Emission Lines', 'Optical Long-Slit Spectroscopy', 'Optical IFU Spectroscopy', 'Hard X-ray Spectroscopy', 'Near-IR IFU Spectroscopy', 'Near-IR Colors', 'Optical Fiber Spectroscopy', 'X-ray Imaging', 'Optical Slitless Spectroscopy', 'Radio Double Sources', 'UV Spectroscopy', 'Submillimeter Imaging', 'Near-IR IFU Imaging', 'Optical Narrow-Band Imaging', 'Radio Imaging', 'X-ray Spectroscopy', 'Near-IR Imaging', 'Mid-IR Spectroscopy', 'UV Imaging', 'Mid-IR Imaging', 'Optical Spectroscopic Emission Line Ratios', 'Hard X-ray Imaging', 'Optical Imaging /Optical Spectroscopy', 'Mid-IR Colors'}
33


In [20]:
# displaying now the instances for each confirmation method

#methods = ['Near-IR Slit Spectroscopy', 'Optical LOS Radial Velocity Shifts', 'Radio-Infrared Colors', 'Optical Narrow-Band Imaging', 'Optical Astrometry', 'Near-IR Periodicity', 'X-ray Periodicity', 'BAT Selection', 'Optical Colors', 'Gamma-Ray Periodicity', 'Mid-IR Imaging', 'Optical Double-Peaked Spectroscopic Emission Lines', 'Optical Photometry', 'Optical Slit Spectroscopy', 'Optical Quasi-Periodicity', 'Optical Spectroscopic Emission Line Ratios', 'Hard X-ray Periodicity', 'Optical Varstrometry', 'Optical Fiber Spectroscopy', 'Positional Offset', 'UV Spectroscopy', 'Radio Periodicity', 'Submillimeter Imaging', 'Radio Double Sources', 'Optical Velocity Offset Broad Spectroscopic Emission Lines', 'X-ray Spectroscopy', 'Optical-Mid-IR Colors', 'Optical IFU Spectroscopy', 'Mid-IR Spectroscopy', 'Optical Periodicity', 'Gamma-Ray Quasi-Periodicity', 'Optical Spectroscopy', 'Modeling', 'Radio X-Shaped Source', 'UV Imaging', 'Near-IR Imaging', 'Infrared Luminosity', 'X-ray Quasi-Periodicity', 'Optical Double-Peaked Broad Spectroscopic Emission Lines', 'Radio-Optical Offsets', 'Near-IR Quasi-Periodicity', 'Radio Double Jet', 'Radio Imaging', 'Optical Velocity Offset Narrow Spectroscopic Emission Lines', 'Serendipitous', 'UV Double-Peaked Narrow Spectroscopic Emission Lines', 'Optical Imaging', 'Optical IFU Imaging', 'X-ray Imaging', 'Optical Slitless Spectroscopy', 'Near-IR Spectroscopy', 'Optical Long-Slit Spectroscopy', 'Mid-IR Colors', 'Near-IR Colors', 'Velocity Offset Emission Lines', 'Radio Quasi-Periodicity', 'Radio Jet Precession']

for i in unique_methods:
    num = 0
    for index,row in MAC.iterrows():
        if i in row['Processed Confirmation Method']:
            num += 1
        else:
            num += 0
    print("Method "+str(i)+" identified "+str(num)+" times!")


# Optical Imaging /Optical Spectroscopy was corrected in individual table. This was under PHL 1222
# Also corrected "  Optical Slit Spectroscopy" to " Optical Slit Spectroscopy"; this was also under PHL 1222

# Near-UV Spectroscopy has been changed to UV Spectroscopy (PHL 1222)
# 
# **** WE NEED TO GO THROUGH AND SEE IF UV SPEC ARE SLIT/LONGSLIT/ETC



Method Optical Imaging identified 33 times!
Method Optical Colors identified 3 times!
Method Optical IFU Imaging identified 3 times!
Method Near-IR Spectroscopy identified 4 times!
Method  Optical Slit Spectroscopy identified 19 times!
Method -99 identified 5520 times!
Method Optical Spectroscopy identified 124 times!
Method Near-UV Spectroscopy identified 1 times!
Method Optical Slit Spectroscopy identified 38 times!
Method Optical Double-Peaked Spectroscopic Emission Lines identified 1 times!
Method Optical Long-Slit Spectroscopy identified 21 times!
Method Optical IFU Spectroscopy identified 4 times!
Method Hard X-ray Spectroscopy identified 1 times!
Method Near-IR IFU Spectroscopy identified 1 times!
Method Near-IR Colors identified 1 times!
Method Optical Fiber Spectroscopy identified 50 times!
Method X-ray Imaging identified 34 times!
Method Optical Slitless Spectroscopy identified 2 times!
Method Radio Double Sources identified 5 times!
Method UV Spectroscopy identified 4 times!

In [21]:
MAC[(MAC['ST1 Confidence Flag']==1) & (MAC['Sep']<6)]

,Unnamed: 0,System Type,Literature Name,Selection Method,Confirmation Method,Name1,z1,z1_type,RA1,Dec1,Coordinate_waveband1,Coordinate_Source1,Equinox1,Brightness1,Brightness_band1,Brightness_type1,Name2,z2,z2_type,RA2,Dec2,Equinox2,Coordinate_waveband2,Coordinate_Source2,Brightness2,Brightness_band2,Brightness_type2,dV,Sep,Sep(kpc),dV_rwp,Paper(s),BibCode(s),DOI(s),Notes,Confidence Flag,Processed System Type,Legacy System Type,System Type 1: Dual AGNs,System Type 2: Binary AGNs,System Type 3: Recoiling AGNs,ST1 Num AGNs,ST2 Num AGNs,ST3 Num AGNs,ST1 Confidence Flag,ST2 Confidence Flag,ST3 Confidence Flag,dV_new,Processed Selection Method,Parsed Selection Method,Analysis Method,Primary System Type,Processed Analysis Method,Processed Confirmation Method
21,21,Dual AGN Candidate / Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Radio Imaging,J005113.94+002047.2,0.11240,spec,00:51:13.94,+00:20:47.2,Optical,SDSS,J2000,-99.0,-99,-99,J005114.12+002049.2,0.112600,spec,00:51:14.12,+00:20:49.2,J2000,Optical,SDSS,-99.0,-99,-99,64.000000,3.360020,6.877886,-53.752809,Liu+2011b ; Fu+2015 ; Fu+2015b ; Gross+2019,2011ApJ...737..101L ; 2015ApJ...799...72F ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,Two compact steep-spectrum sources identified.,1.0,Dual AGN / Dual AGN Candidate,Dual AGN Candidate / Dual AGN,Dual AGN,-99,-99,2,-99,-99,1.0,-99.0,-99.0,-53.752809,Optical Fiber Spectroscopy / Optical Imaging /...,"{'Optical': ['Fiber Spectroscopy', 'Imaging', ...",Fiber Spectroscopy / Optical Spectroscopy / Op...,NaN,Double Radio Sources / Optical Fiber Spectrosc...,Radio Imaging
330,330,Dual AGN Candidate / Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Radio Imaging,J100602.14+071131.0,0.12180,spec,10:06:02.14,+07:11:31.0,Optical,SDSS,J2000,-99.0,-99,-99,J100602.51+071131.8,0.120500,spec,10:06:02.51,+07:11:31.8,J2000,Optical,SDSS,-99.0,-99,-99,354.000000,5.564144,12.192223,346.697471,Liu+2011b ; Ge+2012 ; Rubinur+2019,2011ApJ...737..101L ; 2012ApJS..201...31G ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,. Dual AGN;dual radio sources coincident with...,1.0,Dual AGN / Dual AGN Candidate,Dual AGN Candidate / Dual AGN,Dual AGN,-99,-99,2,-99,-99,1.0,-99.0,-99.0,346.697471,Optical Double-Peaked Spectroscopic Emission L...,{'Optical': ['Double-Peaked Spectroscopic Emis...,Fiber Spectroscopy / Optical Spectroscopy / Op...,NaN,-99 / Optical Double-Peaked Spectroscopic Emis...,Radio Imaging
1030,1030,Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,X-ray Imaging / X-ray Spectroscopy / Optical S...,J154403.45+044607.5,0.04200,spec,15:44:03.45,+04:46:07.5,Optical,SDSS,J2000,-99.0,-99,-99,J154403.67+044610.1,0.041600,spec,15:44:03.67,+04:46:10.1,J2000,Optical,SDSS,-99.0,-99,-99,108.000000,4.192222,3.475073,114.801301,Liu+2011b ; Hou+2019,2011ApJ...737..101L ; 2019ApJ...882...41H,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,LINER+ LINER/Type 2/Composite pair. NE detect...,1.0,Dual AGN,Dual AGN,Dual AGN,-99,-99,2,-99,-99,1.0,-99.0,-99.0,114.801301,Optical Fiber Spectroscopy / Optical Spectrosc...,"{'Optical': ['Fiber Spectroscopy', 'Spectrosco...",Fiber Spectroscopy / Optical Spectroscopy / Op...,NaN,Optical Fiber Spectroscopy / Optical Spectrosc...,Optical Spectroscopy / X-ray Imaging / X-ray S...
1194,1194,Dual AGN Candidate / Dual AGN,-99,Fiber Spectroscopy / Optical Spectroscopy / Op...,Radio Imaging,J220634.97+000327.6,0.04660,spec,22:06:34.97,+00:03:27.6,Optical,SDSS,J2000,-99.0,-99,-99,J220635.08+000323.2,0.046100,spec,22:06:35.08,+00:03:23.2,J2000,Optical,SDSS,-99.0,-99,-99,124.000000,4.699202,4.298385,142.877614,Liu+2011b ; Fu+2015 ; Fu+2015b ; Gross+2019,2011ApJ...737..101L ; 2015ApJ...799...72F ; 20...,https://doi.org/10.1088/0004-637X/737/2/101 ; ...,Two compact steep-spectrum sources identified.,1.0,Dual AGN / Dual AGN Candidate,Dual AGN Candidate / Dual AGN,Dual AGN,-99,-99,2,-99,-99,1.0,-99.0,-99.0,142.877614,Optical Fiber Spectroscopy / Optical Imaging /...,"{'Optical

In [22]:
#MAC.columns

In [23]:
#methods = ['Near-IR Slit Spectroscopy', 'Optical LOS Radial Velocity Shifts', 'Radio-Infrared Colors', 'Optical Narrow-Band Imaging', 'Optical Astrometry', 'Near-IR Periodicity', 'X-ray Periodicity', 'BAT Selection', 'Optical Colors', 'Gamma-Ray Periodicity', 'Mid-IR Imaging', 'Optical Double-Peaked Spectroscopic Emission Lines', 'Optical Photometry', 'Optical Slit Spectroscopy', 'Optical Quasi-Periodicity', 'Optical Spectroscopic Emission Line Ratios', 'Hard X-ray Periodicity', 'Optical Varstrometry', 'Optical Fiber Spectroscopy', 'Positional Offset', 'UV Spectroscopy', 'Radio Periodicity', 'Submillimeter Imaging', 'Radio Double Sources', 'Optical Velocity Offset Broad Spectroscopic Emission Lines', 'X-ray Spectroscopy', 'Optical-Mid-IR Colors', 'Optical IFU Spectroscopy', 'Mid-IR Spectroscopy', 'Optical Periodicity', 'Gamma-Ray Quasi-Periodicity', 'Optical Spectroscopy', 'Modeling', 'Radio X-Shaped Source', 'UV Imaging', 'Near-IR Imaging', 'Infrared Luminosity', 'X-ray Quasi-Periodicity', 'Optical Double-Peaked Broad Spectroscopic Emission Lines', 'Radio-Optical Offsets', 'Near-IR Quasi-Periodicity', 'Radio Double Jet', 'Radio Imaging', 'Optical Velocity Offset Narrow Spectroscopic Emission Lines', 'Serendipitous', 'UV Double-Peaked Narrow Spectroscopic Emission Lines', 'Optical Imaging', 'Optical IFU Imaging', 'X-ray Imaging', 'Optical Slitless Spectroscopy', 'Near-IR Spectroscopy', 'Optical Long-Slit Spectroscopy', 'Mid-IR Colors', 'Near-IR Colors', 'Velocity Offset Emission Lines', 'Radio Quasi-Periodicity', 'Radio Jet Precession']
#
#
#
#for i in methods:
#    num = 0
#    for index,row in MAC.iterrows():
#        if i in row['Processed Selection Method']:
#            num += 1
#        else:
#            num += 0
#    print("Method "+str(i)+" identified "+str(num)+" times!")


# Now generating the parsed, nested dictionaries for the analysis and confirmation method columns

In [24]:
# now here we'll format the analysis and confirmation method columns, converting them into nested dictionaries 

import re

def parse_selection_method(s, wavebands, compound_wavebands):
    # Split the string by slashes
    methods = s.split(' / ')

    result_dict = {}
    for method in methods:
        # Explicitly handle 'Optical-Mid-IR Colors'
        if 'Optical-Mid-IR Colors' in method:
            for wb in ['Optical', 'Mid-IR']:
                if wb in result_dict:
                    result_dict[wb].append('Optical-Mid-IR Colors')
                else:
                    result_dict[wb] = ['Optical-Mid-IR Colors']
        else:
            # First check for compound wavebands
            compound_processed = False
            for compound in compound_wavebands:
                if compound in method:
                    term = method.replace(compound, '').strip()
                    for wb in compound.split('-'):
                        if wb in result_dict:
                            result_dict[wb].append(compound + ' ' + term)
                        else:
                            result_dict[wb] = [compound + ' ' + term]
                    compound_processed = True
                    break
            
            if not compound_processed:
                # Process standard wavebands
                for waveband in wavebands:
                    if waveband in method:
                        term_start_idx = method.index(waveband) + len(waveband)
                        term = method[term_start_idx:].strip()

                        if waveband in result_dict:
                            result_dict[waveband].append(term)
                        else:
                            result_dict[waveband] = [term]

    return result_dict

# List of known wavebands
known_wavebands = ['Optical', 'Mid-IR', 'Radio', 'Hard X-ray', 'X-ray', 'UV', 'Infrared', 'Submillimeter',\
                   'Near-IR','Gamma-Ray']

# List of compound wavebands
compound_wavebands = ['Radio-Optical', 'Optical-Mid-IR', 'Radio-Infrared']

# Apply the function to each row
MAC['Parsed Analysis Method'] = MAC['Processed Analysis Method'].apply(lambda x: parse_selection_method(x, known_wavebands, compound_wavebands))
MAC['Parsed Confirmation Method'] = MAC['Processed Confirmation Method'].apply(lambda x: parse_selection_method(x, known_wavebands, compound_wavebands))




In [25]:
import ast

def get_selection_method(waveband, technique):
    def filter_function(row):
        # Access the value in the 'Parsed Selection Method' column
        selection_method = row['Parsed Selection Method']
        
        # If the value is a string, convert it back to a dictionary
        if isinstance(selection_method, str):
            try:
                selection_dict = ast.literal_eval(selection_method)
            except ValueError:
                return False
        else:
            selection_dict = selection_method

        # Check if the waveband is a key and the technique is one of its values
        return str(waveband) in selection_dict and str(technique) in selection_dict[str(waveband)]

    return filter_function

def get_analysis_method(waveband, technique):
    def filter_function(row):
        # Access the value in the 'Parsed Selection Method' column
        analysis_method = row['Parsed Analysis Method']
        
        # If the value is a string, convert it back to a dictionary
        if isinstance(analysis_method, str):
            try:
                analysis_dict = ast.literal_eval(analysis_method)
            except ValueError:
                return False
        else:
            analysis_dict = analysis_method

        # Check if the waveband is a key and the technique is one of its values
        return str(waveband) in analysis_dict and str(technique) in analysis_dict[str(waveband)]

    return filter_function

## Apply the filter function with specific waveband and technique
#filter_selection = get_selection_method('Optical', 'Spectroscopy')
#df_selection = MAC[MAC.apply(filter_selection, axis=1)]
#


In [26]:
MAC.to_csv('MAC_DR0p9.csv')

In [28]:
MAC = MAC.drop(['Processed Selection Method','Analysis Method','Processed Analysis Method',\
                 'Processed Confirmation Method',\
                 'Notes','Processed System Type','Confidence Flag','Selection Method',\
                 'Confirmation Method','Unnamed: 0','System Type'], axis=1)


MAC.to_csv('MAC_dr0p9_formatting.csv')

In [ ]:
from astropy.table import Table


MAC1 = MAC.drop(['Processed Selection Method','Analysis Method','Processed Analysis Method',\
                 'Processed Confirmation Method',\
                 'Notes','Confirmation Methood','Processed System Type','Confidence Flag','Selection Method',\
                 'Confirmation Method','Unnamed: 0','System Type'], axis=1)
t = Table.from_pandas(MAC1)

t.write('MAC_DR0p9_beta_webtable_7Feb2024.html', format='jsviewer', overwrite=True)


